In [ ]:
import * as dotenv from 'npm:dotenv';
import base64 from 'npm:base-64';  // Use default import instead of namespace import

dotenv.config();

const LANGFUSE_PUBLIC_KEY = "pk-lf-...";
const LANGFUSE_SECRET_KEY = "sk-lf-...";
const LANGFUSE_AUTH = base64.encode(`${LANGFUSE_PUBLIC_KEY}:${LANGFUSE_SECRET_KEY}`);


process.env["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://cloud.langfuse.com/api/public/otel"; // EU data region
// process.env["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel"; // US data region
process.env["OTEL_EXPORTER_OTLP_HEADERS"] = `Authorization=Basic ${LANGFUSE_AUTH}`;


process.env["TRACELOOP_BASE_URL"] = "https://cloud.langfuse.com/api/public/otel"; // EU data region
// process.env["OTEL_EXPORTER_OTLP_ENDPOINT"] = "https://us.cloud.langfuse.com/api/public/otel"; // US data region
process.env["TRACELOOP_HEADERS"] = `Authorization=Basic ${LANGFUSE_AUTH}`;

// your openai key
process.env["OPENAI_API_KEY"] = "sk-...";

In [3]:
// docs: https://ts.llamaindex.ai/docs/llamaindex/integration/open-llm-metry

import * as traceloop from "npm:@traceloop/node-server-sdk";

traceloop.initialize({ disableBatch: true });

Traceloop exporting traces to https://cloud.langfuse.com/api/public/otel


In [9]:
// docs: https://ts.llamaindex.ai/docs/llamaindex/integration/lang-trace

import * as Langtrace from "npm:@langtrase/typescript-sdk";

Langtrace.init();

@opentelemetry/api: Registered a global for diag v1.9.0.


@langtrase/typescript-sdk Module cohere-ai has been loaded before @langtrase/typescript-sdk so it might not work, please initialize it before requiring cohere-ai
@langtrase/typescript-sdk Module @google-cloud/vertexai has been loaded before @langtrase/typescript-sdk so it might not work, please initialize it before requiring @google-cloud/vertexai


: 

In [4]:
import { OpenAI } from "npm:llamaindex";

const llm = new OpenAI();
const response = await llm.chat({
  messages: [{ content: "Tell me a joke.", role: "user" }],
});

console.log(response.message.content);

Why don't skeletons fight each other?

They don't have the guts!


In [5]:
import OpenAI from "npm:openai";
const openai = new OpenAI();

const completion = await openai.chat.completions.create({
    model: "gpt-4o-mini",
    messages: [
        { role: "system", content: "You are a helpful assistant." },
        {
            role: "user",
            content: "Write a haiku about recursion in programming.",
        },
    ],
    store: true,
});

console.log(completion.choices[0].message);

{
  role: "assistant",
  content: "Functions call themselves,  \n" +
    "Layers of logic unfold—  \n" +
    "Endless loops of thought.",
  refusal: null
}
